In [ ]:
import glob, pandas as pd, geopandas as gpd, numpy as np, pyproj, cameratransform as ct, warnings, os
from statistics import mean
from shapely.geometry import Point, Polygon
from scipy.spatial.transform import Rotation
from osgeo import gdal, osr

# set the base directory where everything lives
base_dir = r"U:\SOC_2022\IRMA"
# and where outputs will be nested
out_dir = base_dir

# these should be self-explanatory
# script is currently set up to only run one survey at a time, but could be easily looped if necessary
target_year = 2022
target_survey_type = "Random"
# alternatives are "Optimal", "Random", or "Abundance"

# name of the CSV file within the base directory
# This script was developed to handle the 2022 year of data as available on NPS IRMA: https://doi.org/10.57830/2302816
CSV_file = "SO_D_2017-2022.CSV"

# This variable links generic column names (left side) to the specific column names (right side) in the CSV that we're using
# An input CSV file must have at least the first set of columns (before the break), the rest are generated in the script
# This dictionary is here to allow users to easily substitute their own column names

cols = {"Name":"PHOTO_FILE_NAME",
        "OriginalName": "ORIGINAL_FILENAME",
        "TimeStamp(UTC)":"PHOTO_TIMESTAMP_UTC",
        "SurveyType":"SURVEY_TYPE",
        "Latitude":"LATITUDE_WGS84",
        "Longitude":"LONGITUDE_WGS84",
        "Altitude": "ALTITUDE",
        "Camera": "CAMERA_SYSTEM",

        "SurveyYear":"YEAR",
        "Heading": "HEADING",
        "Yaw": "HEADING",
        "Pitch": "PITCH_ANGLE",
        "Roll": "ROLL_ANGLE",
        "GCPs": "GCP_LIST",
        "Notes": "NOTES AREA CALC"}

# read in CSV database, subset by year and survey type, create column in which to record data notes
df = pd.read_csv(f"{base_dir}\\{CSV_file}", keep_default_na=False, low_memory=False)
df[cols['SurveyYear']] = df[cols['TimeStamp(UTC)']].str.split('-').str[0]
df = df[df[cols['SurveyYear']] == str(target_year)]
df = df[df[cols['SurveyType']] == str(target_survey_type)]
df[cols['Notes']] = ""
df = df.reset_index(drop=True)

In [ ]:
# if you want to subset the data, this is a good time to do it (or just subset the CSV loading)
#df

In [ ]:
# some variables to help set up the camera parameters
cameras = list(set(df[cols['Camera']]))

sensor_width_dictionary = {"Sony a6100": 23.5,
                      "Canon EOS 5DS R": 36,
                     "NIKON D810": 35.9}
if len(cameras) > 1:
    print(f"Multiple camera systems detected: {cams}. That's not currently supported by this script.")

# for the Waldo system, split the datasets into left- and right-side photographs and assign the appropriate camera model
# otherwise we're using just a single dataframe

# This script does not currently resort images because the "TimeStamp" column seems to resolve only to "minute" in 2022,
# so many images share the same timestamp. Name, likewise, is not entirely sequential, alternating between left and right cameras.
# If sorting is wanted, I recommend to (1) split the dataframes by left/right (original name) then (2) sort by new name
# Note: second-resolved timestamps are present in the original photos and seem to have been scrubbed during the processing
# so they are not present in IRMA but are recoverable

if cameras == ['WALDO XCAMUltra50']:
    # Note: "Right" and "Left" were manually assigned based on the orientation of imagery relative to the aircraft's heading
    dfs = {"Right": df[df[cols['OriginalName']].str[0] == "0"], "Left": df[df[cols['OriginalName']].str[0] == "1"]}
    camera_model = "Canon EOS 5DS R"
    waldo_angle = 39.6
else:
    dfs = {"Single": df}
    print("This script is currently designed to handle WALDO cameras, other cameras likely require script modifications")

# setting up camera parameters for later image projection method
# these values can be pulled from a sample image's exif data, but I'm hard-coding here for simplicity's sake
sensor_w = sensor_width_dictionary[camera_model]
focal_length = 50
orig_image_size = orig_image_w, orig_image_h = 8688, 5792

orig_aspect_ratio = orig_image_w/orig_image_h
sensor_h = sensor_w * orig_aspect_ratio

cr_image_size = cr_image_w, cr_image_h = orig_image_w-250, orig_image_h-250
w_reduction, h_reduction = cr_image_w/orig_image_w, cr_image_h/orig_image_h
cr_sensor_size = cropped_sensor_w, cropped_sensor_h = sensor_w*w_reduction, sensor_h*h_reduction
fov_w = 2 * np.degrees(np.arctan((0.5*cropped_sensor_w)/focal_length))

# The following parameters are options for "plug and chug" experimentation to see if it helps align imagery

# This parameter is an offset (meters) to add to the altitude, considering that GPS altitude often doesn't align with true distance from sea level
altitude_offset = 0
# This parameter is an offset (degrees) representing the potential "off-center" swing of the WALDO system, in case it was slightly tilted to one side or the other
swing_offset = 0
# This parameter was set earlier, but could be modified too
waldo_angle = waldo_angle

In [ ]:
# This function calculates the camera's heading based on the camera's trajectory across current, previous, and following locations
# Headings are weighed proportional to the distances of these locations, but could be improved to account for likely longer
# trajectory across longer distances. A more accurate heading estimate could be pulled from the complete raw photo set.
# This is not specifically needed if the camera metadata includes orientation (pitch/roll/yaw) values

# import pyproj
# from statistics import mean
geodesic = pyproj.Geod(ellps='WGS84')

def generate_headings(df, cols=cols):
    headings = {}
    df = df.reset_index(drop=True)
    for i in range(min(df.index),max(df.index)+1):
        azims=[None, None]
        dists=[None, None]
        long1, lat1 = df[cols['Longitude']][i], df[cols['Latitude']][i]
        if i > min(df.index):
            long0, lat0 = df[cols['Longitude']][i-1], df[cols['Latitude']][i-1]
            if (long0 != long1) and (lat0 != lat1): # handling for if two consecutive photos have the same GPS location
                azims[0],dists[0] = np.array(geodesic.inv(long0, lat0, long1, lat1))[[0, 2]]
            else:
                azims[0], dists[0] = headings[df[cols['Name']][i-1]], 0 # assign azimuth from previous image
        if i < max(df.index):
            long2, lat2 = df[cols['Longitude']][i+1], df[cols['Latitude']][i+1]
            if (long1 != long2) and (lat1 != lat2): # handling for if two consecutive photos have the same GPS location
                azims[1],dists[1] = np.array(geodesic.inv(long1, lat1, long2, lat2))[[0, 2]]
            else: dists[1] = 0
        if not (None in azims+dists):
            if abs(azims[0]-azims[1]) < 45:
                d1_proportion, d2_proportion = 1-(dists/(sum(dists)))
                heading = (d1_proportion * azims[0]) + (d2_proportion * azims[1])
            else:
                heading = azims[dists.index(min(dists))]
        else:
            heading = mean(azim for azim in azims if azim is not None)
        headings[df[cols['Name']][i]] = heading
    return(headings)

# I don't love this chain of if conditions but it works for now.
# Could probably create a better heading estimator using more sophisticated interpolation methods
# Or by pulling from the complete photo set (which includes turn-around photos between transects)

In [ ]:
# This section iterates through the dataframes and generates headings using our headings function
# it assigns headings based on the image name, so as not to rely on index values, which could get changed
# This section also assigns default pitch roll values, with offsets for the Waldo cameras
# The Waldo cameras are fixed 39.6° apart, and we assume that they are perfectly centered (+/- 19.8 degrees)
# (we also assume that the aircraft is perfectly level at all times)
# Future survey efforts can/should include inclinometer data if possible

for side, frame in dfs.items():
    if cameras == ['WALDO XCAMUltra50']:
        roll_angle = waldo_angle/2
        if side == "Right": df.loc[df[cols['Name']].isin(frame[cols['Name']]), cols['Heading']]= df[cols['Name']].map(generate_headings(frame))
        elif side == "Left": df.loc[df[cols['Name']].isin(frame[cols['Name']]), cols['Heading']]= df[cols['Name']].map(generate_headings(frame))+180
        # I use a reverse heading instead of a reverse roll because that's what works
        # when rotation notation is converted from ZXY to ZXZ, I can't say why
    else:
        roll_angle = 0
        df.loc[df[cols['Name']].isin(frame[cols['Name']]), cols['Heading']]= df[cols['Name']].map(generate_headings(frame))

df[cols['Pitch']] = 0
df[cols['Roll']] = roll_angle
# clean up any angles that include a full revolution
df.loc[df[cols['Heading']]>360, cols['Heading']] = df.loc[df[cols['Heading']]>360][cols['Heading']]-360
df[cols['Notes']] += "Heading calculated from consecutive image locations. Pitch assumed to be 0, Roll assumed to be 0, Camera Roll assumed to be +- 19.8°, Yaw set to Heading. "


# apply swing offset (if any)
df.loc[df[cols['OriginalName']].str[0] == "0", cols['Roll']] = df.loc[df[cols['OriginalName']].str[0] == "0"][cols['Roll']]+swing_offset
df.loc[df[cols['OriginalName']].str[0] == "1", cols['Roll']] = df.loc[df[cols['OriginalName']].str[0] == "1"][cols['Roll']]-swing_offset

# This section could certainly be cleaned up, but it works for now

In [ ]:
# This section estimates photograph locations based on camera position
# Some conversion is needed to get our available/assumed position information (pitch, roll, yaw)
# into the format that the cameratransform function can use (ZXY to ZXZ expression of intrinsic rotation angles)

# import cameratransform as ct
# from shapely.geometry import Point, Polygon
# from scipy.spatial.transform import Rotation

# set the camera projection
rlp = ct.RectilinearProjection(focallength_mm = focal_length, sensor = cr_sensor_size, image = cr_image_size)
df[cols['GCPs']] = df['geometry'] = pd.Series(dtype='object')

for i in df.index:
    # transform the rotation values
    zxz_Heading, zxz_Tilt, zxz_Roll = Rotation.from_euler('ZXY' ,[df.loc[i, cols['Heading']],
                                                                  df.loc[i, cols['Pitch']],
                                                                  df.loc[i, cols['Roll']]
                                                                 ], degrees=True).as_euler('ZXZ', degrees=True)

    # can't say why this is necessary either. Maybe something to do with the orthogonal orientation of the drone axes vs. camera?
    zxz_Tilt, zxz_Roll = -1*zxz_Tilt, -1*zxz_Roll 
    
    # set the spatial orientation of the camera
    cam = ct.Camera(rlp, ct.SpatialOrientation(elevation_m = df.loc[i, cols['Altitude']], tilt_deg = zxz_Tilt, 
                                               roll_deg = zxz_Roll, heading_deg = zxz_Heading, 
                                               pos_x_m = 0, pos_y_m = 0))
    # and the GPS location
    cam.setGPSpos(float(df.loc[i, cols['Latitude']]), float(df.loc[i, cols['Longitude']]), altitude_offset+float(df.loc[i, cols['Altitude']]))

    # make a list of image coordinates that we'll use as GCPs
    img_edgepoints = ([0, 0], # top left
                  [cr_image_w-1, 0], # top right
                  [cr_image_w-1, cr_image_h-1], # bottom right
                  [0, cr_image_h-1], # bottom left
                  
                  [(cr_image_w-1)/2, 0], # top midpoint
                  [cr_image_w-1, (cr_image_h-1)/2], # right midpoint
                  [(cr_image_w-1)/2, cr_image_h-1], # bottom midpoint
                  [0, (cr_image_h-1)/2] # left midpoint
                     )
    # convert the image coordinates to GCPs
    GCP_list = [cam.gpsFromImage(i).tolist()[0:2][::-1] + [0] + i for i in img_edgepoints]
    
    # assign values to dataframe as GCPs for image warping and as polygon geometry for a shapefile
    df.loc[i, cols['GCPs']] = [GCP_list]
    df.loc[i,'geometry'] = Polygon([i[0:2] for i in GCP_list][0:4])

In [ ]:
# This section outputs the geodataframe to a shapefile with all columns as attributes.
# Note that attribute field names get truncated to 10 characters, a warning will complain about this

# This section suppresses those warnings
# import warnings, os
warnings.filterwarnings("ignore", message="Normalized/laundered field name")

directory_suffix = f"_{target_year}_{target_survey_type}_{altitude_offset}alt_{swing_offset}sw"

gdf=gpd.GeoDataFrame(df, crs="EPSG:4326")
del gdf[cols['Notes']], gdf[cols['GCPs']]
out_path = f"{out_dir}\\Shapefiles{directory_suffix}\\Image_footprints_{target_year}{target_survey_type}.shp"
os.makedirs(f"{out_path.replace(out_path.split("\\")[-1], "")}", exist_ok = True)
gdf.to_file(out_path)

In [ ]:
# from osgeo import gdal, osr

# set this to True if you want to project the photographs into spatial coordinates
warp_photographs = False

# alternative format "GeoTIFF" is much larger per file
out_format = "JPEG"

# directory where the source photographs live
img_dir = r"U:\SOC_2022\IRMA\20220812_R"

if warp_photographs == True:
    out_path = f"{out_dir}\\georeferenced{directory_suffix}\\"
    os.makedirs(f"{out_path}", exist_ok = True)

    gdf=gpd.GeoDataFrame(df, crs="EPSG:4326")
    gdf=gdf.loc[1691:1694]
    for index, record in gdf.iterrows():
    
        img_path = f"{img_dir}\\{record[cols['Name']]}"
        
        # Read in the image file:
        ds = gdal.Open(img_path)
        if ds is None:
            print(f"Could not open image: {img_path}")
        # Set spatial reference:
        sr = osr.SpatialReference()
        sr.ImportFromEPSG(4326)
    
        # Import GCPs
        gcps = [gdal.GCP(*i) for i in record['GCP_LIST'][0][0:7]]
    
        # Apply the GCPs to the open output file then warp it
        ds.SetGCPs(gcps, sr.ExportToWkt())
    
        if out_format=="JPEG":
            kwargs = {'format': 'JPEG', 'polynomialOrder':2, 'srcNodata': '0,0,0', 'dstNodata': 'nodata'}
            ds = gdal.Warp(f"{out_path}\\{record[cols['Name']]}_GeoRef.jpg", ds, **kwargs)
        elif out_format=="GeoTIFF":
            kwargs = {'format': 'GTiff', 'polynomialOrder':2, 'srcNodata': '0,0,0', 'dstNodata': 'nodata'}
            ds = gdal.Warp(f"{out_path}\\{record[cols['Name']]}_GeoRef.tif", ds, **kwargs)
        else:
            print(fr"format '{out_format}' not currently supported, code it yourself from https://gdal.org/en/latest/drivers/raster/index.html")
            break
        # Clear the variable (not sure if necessary, but good form)
        ds = None
        # counter, just because it can be a long process
        if (index+1) % 10 == 0:
            print(f"Completed file {index+1} out of {len(df)+1}")
    print("finished processing images")